In [1]:
import torch
import torch.nn as nn
import numpy as np
torch.__version__

'1.1.0'

In [4]:
# 1. 加载数据
# 二分类的数据集
data=np.loadtxt('german.data-numeric')
print(data.shape)

(1000, 25)


In [5]:
# 2. 数据归一化处理
r,c=data.shape
for j in range(c-1):
    mean_val=np.mean(data[:,j])
    std_val=np.std(data[:,j])
    data[:,j]=(data[:,j]-mean_val)/std_val
print(data)

[[-1.25456565 -1.23647786  1.34401408 ... -0.5         0.76635604
   1.        ]
 [-0.45902624  2.24819436 -0.50342796 ... -0.5         0.76635604
   2.        ]
 [ 1.13205258 -0.73866754  1.34401408 ...  2.         -1.30487651
   1.        ]
 ...
 [ 1.13205258 -0.73866754 -0.50342796 ... -0.5         0.76635604
   1.        ]
 [-1.25456565  1.9992892  -0.50342796 ... -0.5         0.76635604
   2.        ]
 [-0.45902624  1.9992892   1.34401408 ... -0.5         0.76635604
   1.        ]]


In [8]:
# 打乱数据
np.random.shuffle(data) # 原地操作，行行之前打乱
#data= np.random.permutation(data)# 返回一个修改后的数组
print(data)

[[ 1.13205258  0.25695309 -0.50342796 ... -0.5         0.76635604
   1.        ]
 [-0.45902624 -0.07492045  0.42029306 ... -0.5        -1.30487651
   1.        ]
 [ 1.13205258  0.25695309  1.34401408 ... -0.5        -1.30487651
   1.        ]
 ...
 [ 0.33651317 -0.9875727  -0.50342796 ...  2.         -1.30487651
   2.        ]
 [-1.25456565  2.24819436  1.34401408 ...  2.         -1.30487651
   2.        ]
 [-1.25456565  0.25695309 -1.42714898 ... -0.5         0.76635604
   1.        ]]


In [10]:
a=[[1,2],[3,4],[5,6]]
np.random.shuffle(a) # shuffle是原地操作
#a= np.random.permutation()# 返回一个修改后的数据

[[1, 2], [5, 6], [3, 4]]

In [33]:
# 生成训练集和测试集
train_data=data[:900,:c-1]
train_lab=data[:900,c-1]-1
test_data=data[900:,:c-1]
test_lab=data[900:,c-1]-1

In [65]:
# 定义模型
class LR(nn.Module):
    def __init__(self):
        super(LR,self).__init__()
        #self.fc=nn.Linear(24,2) #2为类别数目
        self.fc1=nn.Linear(24,100)
        self.fc2=nn.Linear(100,2)
    def forward(self,x):
        out=self.fc1(x)
        out=self.fc2(out)
        out=torch.sigmoid(out)
        return out

In [66]:
net=LR()
criterion=nn.CrossEntropyLoss() #使用CrossEntropyLoss计算损失
optim=torch.optim.Adam(net.parameters()) # Adam优化
epochs=1000

In [67]:
def test(pred,lab):
    print(pred.shape,lab.shape)
    #print(pred.max(-1)[1])
    t=pred.max(1)[1]==lab
    #print(t)
    return torch.mean(t.float())
for i in range(epochs):
    # 指定模型为训练模式，计算梯度
    net.train()
    x=torch.from_numpy(train_data).float()
    y=torch.from_numpy(train_lab).long()
    y_hat=net(x)
    if (i+1)%100==0:
        #print(y_hat,y)
        pass
    loss=criterion(y_hat,y)
    optim.zero_grad()
    loss.backward() # 反向传播
    optim.step() #优化
    if (i+1)%100==0:
        # 评估模型
        net.eval()
        test_in=torch.from_numpy(test_data).float()
        test_target=torch.from_numpy(test_lab).long()
        test_out=net(test_in)
        # 使用我们的测试函数计算准确率
        accu=test(test_out,test_target)
        print("Epoch:{},Loss:{:.4f},Accuracy：{:.2f}".format(i+1,loss.item(),accu))

torch.Size([100, 2]) torch.Size([100])
Epoch:100,Loss:0.5306,Accuracy：0.72
torch.Size([100, 2]) torch.Size([100])
Epoch:200,Loss:0.5134,Accuracy：0.74
torch.Size([100, 2]) torch.Size([100])
Epoch:300,Loss:0.5087,Accuracy：0.73
torch.Size([100, 2]) torch.Size([100])
Epoch:400,Loss:0.5027,Accuracy：0.72
torch.Size([100, 2]) torch.Size([100])
Epoch:500,Loss:0.4987,Accuracy：0.72
torch.Size([100, 2]) torch.Size([100])
Epoch:600,Loss:0.4959,Accuracy：0.73
torch.Size([100, 2]) torch.Size([100])
Epoch:700,Loss:0.4936,Accuracy：0.73
torch.Size([100, 2]) torch.Size([100])
Epoch:800,Loss:0.4916,Accuracy：0.73
torch.Size([100, 2]) torch.Size([100])
Epoch:900,Loss:0.4898,Accuracy：0.73
torch.Size([100, 2]) torch.Size([100])
Epoch:1000,Loss:0.4883,Accuracy：0.73


In [59]:
nn.CrossEntropyLoss()(torch.FloatTensor([[0, 1]]),torch.LongTensor([1]))
torch.max(torch.FloatTensor([[0, 1,2],[3,4,5]]),0) # [3,4,5],按第0个维度(行)取最大值
torch.max(torch.FloatTensor([[0, 1,2],[3,4,5]]),1) # [2,5],按第1个维度(列)取最大值


torch.return_types.max(
values=tensor([2., 5.]),
indices=tensor([2, 2]))